In [11]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =10

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

In [12]:
os.chdir("data")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'data'

In [13]:
df = pd.read_csv("CleanedData.csv")

In [14]:
df["Category_name"].value_counts()

tv, television, led                2266
drive, storage, hard               2105
headphone, earbud, bluetooth       1801
tv, 4k, television                 1636
headphone                          1580
receiver, amplifier, home          1541
car, gps, dash                     1464
camera, shoot, point               1386
tablet, computer, ipad             1363
camera, mirrorless                 1302
laptop, computer                   1210
speaker, portable, bluetooth       1202
player, ipod                       1098
sound, speaker, home               1088
car, speaker, subwoofer            1072
speaker, audio                     1035
case, computer                      926
laptop, computer, macbook           925
speaker, bluetooth, mobile          841
phone, cell                         789
mount, tv, stand                    783
camera, camcorder, action           750
wireless, speaker, audio            696
lens, camera                        683
monitor, computer                   654


Model for "Speakers"

In [15]:
dfx = df[df["Category_name"].str.contains("speaker",case=False)]
dfx= dfx[dfx["Year"]==2017]

In [16]:
dfx = dfx.reset_index(drop=True)

In [17]:
dfx.sort_values(by=['Date'],ignore_index=True)

,id,Name,Brand,Description,Category_name,Merchant,Actual_price,Discount_price,isSale,Discount_per,Imp_count,Day,Month,Year,Week,Month_Name,Day_Name,Date
0,AVphKa...,SRS-ZR...,Sony,Stereo...,wirele...,bhphot...,198.00,198.00,No,0.0,2,28,2,2017,9,February,Tuesday,2017-0...
1,AVpgcA...,Sony -...,Sony,Stereo...,speake...,bhphot...,238.00,238.00,No,0.0,2,28,2,2017,9,February,Tuesday,2017-0...
2,AVphS0...,Sony -...,Sony,Audio ...,speake...,bhphot...,348.00,348.00,No,0.0,3,1,3,2017,9,March,Wednesday,2017-0...
3,AVpfHc...,SP-FS5...,Pioneer,Speake...,speake...,Bestbu...,129.99,129.99,No,0.0,13,2,3,2017,9,March,Thursday,2017-0...
4,AVphe2...,Refere...,Klipsch,Stereo...,speake...,Bestbu...,549.98,549.98,No,0.0,19,2,3,2017,9,March,Thursday,2017-0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6156,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,30,12,2017,52,December,Saturday,2017-1...
6157,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...
6158,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...
6159,AVpgoU...,JBL Cl...,JBL,Stereo...,speake...,kohls.com,59.99,59.99,No,0.0,46,31,12,2017,52,December,Sunday,2017-1...


In [28]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector

X, y = load_iris(return_X_y=True)

# Create a logistic regression model
lr = LogisticRegression()

sfs = SequentialFeatureSelector(lr, n_features_to_select="auto")
# Perform forward selection
sfs.fit(X, y)

SequentialFeatureSelector(estimator=lr,
                          n_features_to_select=3)

# Print the selected features
print('Selected features:', sfs.get_support(indices=True))


TypeError: __init__() got an unexpected keyword argument 'n_features_to_select'

In [19]:
import sklearn

In [20]:
(sklearn.__version__)

'1.2.1'

In [42]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=3)
sfs.fit(X, y)
SequentialFeatureSelector(estimator=LogisticRegression, n_features_to_select=3)
sfs.get_support()
np.array([ True, False,  True,  True])
sfs.transform(X).shape

(150, 3)

In [43]:
print('Selected features:', sfs.get_support(indices=True))

Selected features: [0 2 3]
